In [1]:
import os
from pprint import pprint

Запустить из консоли

opus_read --directory OpenSubtitles --source en --target ru -m 900000 --write result.txt --src_range 1-4 --tgt_range 1-4

https://github.com/Helsinki-NLP/OpusTools

In [2]:
with open("subtitles_raw/ru-en_0-900.txt", "rt", encoding="utf-8") as f:
    text = f.read()

data = text.split('================================')

In [3]:
data = [item.strip() for item in data]

ru = []
en = []
idx = 0
for item in data:
    if item.startswith('#') or not item:
        continue
    sentences = item.split('\n')
    src = []
    trg = []
    for sentence in sentences:
        # print(idx, sentence)
        if sentence.startswith("(src)"):
            trash, useful = sentence.split(">");
            useful = useful.split();
            src += useful
        else:
            trash, useful = sentence.split(">");
            useful = useful.split();
            trg += useful
        idx += 1
        
    ru.append(src)
    en.append(trg)


In [4]:
pprint(ru[:3])

[['Дети',
  'могут',
  'достать',
  'во',
  'дворе',
  'почти',
  'всё',
  'что',
  'угодно',
  'до',
  'тех',
  'пор',
  ',',
  'пока',
  'могут',
  'себе',
  'это',
  'позволить',
  '.'],
 ['Всё', 'имеет', 'свою', 'цену', '.'],
 ['Эй', ',', 'Ник', '.']]


In [5]:
pprint(en[:3])

[['Kids',
  'can',
  'get',
  'pretty',
  'much',
  'anything',
  'they',
  'want',
  'in',
  'the',
  'yard',
  ',',
  'as',
  'long',
  'as',
  'they',
  'can',
  'afford',
  'it',
  '.'],
 ["'Cause", 'everything', 'comes', 'with', 'a', 'price', '.'],
 ['Hey', ',', 'Nick', '.']]


In [6]:
import spacy
from tqdm import tqdm
from simalign import SentenceAligner

In [7]:
myaligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-04-13 21:57:28,681 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [11]:
en_nlp_md = spacy.load("en_core_web_md")
ru_nlp_md = spacy.load("ru_core_news_md")

In [ ]:
chunk_size = 20000
latest_chunk = 320000
for chunk_start in range(latest_chunk, len(ru), chunk_size):
    mwmf = ""
    itermax = ""
    inter = ""



    for i in tqdm(range(chunk_start, chunk_start + chunk_size)):
        ru_sentence = " ".join(ru[i])
        ru_doc = ru_nlp_md(ru_sentence)
        ru_sentence = " ".join(ru[i])
        ru_doc = ru_nlp_md(ru_sentence)

        src, trg = (ru[i], en[i])
        alignments = myaligner.get_word_aligns(src, trg)
        mwmf += " ".join([f"{x}-{y}" for x, y in alignments["mwmf"]])
        itermax += " ".join([f"{x}-{y}" for x, y in alignments["itermax"]])
        inter += " ".join([f"{x}-{y}" for x, y in alignments["inter"]])
        mwmf += "\n"
        itermax += "\n"
        inter += "\n"

    folder_name = "corpora/subtitles/numerical_alignment"
    filename = f"aligned_{chunk_start}-{chunk_start + chunk_size - 1}"
    fname_mwmf = f"{folder_name}/mwmf/{filename}"
    fname_itermax = f"{folder_name}/itermax/{filename}"
    fname_inter = f"{folder_name}/inter/{filename}"

    with open(fname_mwmf, 'wt', encoding="utf-8") as f:
        f.write(mwmf)

    with open(fname_itermax, 'wt', encoding="utf-8") as f:
        f.write(itermax)

    with open(fname_inter, 'wt', encoding="utf-8") as f:
        f.write(inter)

 35%|███▍      | 6960/20000 [12:22<33:37,  6.46it/s]  

In [19]:
п

In [20]:
for word in doc:
    print(f"|{word.text}|, {word.lemma_}, {word.pos_}")

|Kids|, kid, NOUN
|can|, can, AUX
|get|, get, VERB
|pretty|, pretty, ADV
|much|, much, ADJ
|anything|, anything, PRON
|they|, they, PRON
|want|, want, VERB
|in|, in, ADP
|the|, the, DET
|yard|, yard, NOUN
|,|, ,, PUNCT
|as|, as, ADV
|long|, long, ADV
|as|, as, SCONJ
|they|, they, PRON
|can|, can, AUX
|afford|, afford, VERB
|it|, it, PRON
|.|, ., PUNCT
